In [ ]:
from torchvision import transforms, datasets
import torch
from torch import nn
import zipfile

NameError: name '_C' is not defined

In [ ]:
from torchvision import transforms, datasets
import torch
from torch import nn
import zipfile

NameError: name '_C' is not defined

In [ ]:
class classificador(nn.Module):
  def __init__(self):
    super().__init__()
    self.activation = nn.ReLU()
    self.pool = nn.MaxPool2d(kernel_size = (2,2))
    self.bnorm = nn.BatchNorm2d(num_features = 32)
    self.flatten = nn.Flatten()
    self.dropout = nn.Dropout(0.2)
    self.output = nn.Sigmoid()

    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = (3,3))
    self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = (3,3))

    self.linear1 = nn.Linear(in_features = 14*14*32, out_features = 128)
    self.linear2 = nn.Linear(in_features = 128, out_features = 128)
    self.linear3 = nn.Linear(in_features = 128, out_features = 1)

  def forward(self, x):
    x = self.pool(self.bnorm(self.activation(self.conv1(x))))
    x = self.pool(self.bnorm(self.activation(self.conv2(x))))

    x = self.flatten(x)

    x = self.dropout(self.activation(self.linear1(x)))
    x = self.dropout(self.activation(self.linear2(x)))
    x = self.output(self.linear3(x))
    return x

NameError: name 'nn' is not defined

In [ ]:
criterion = nn.BCELoss()
net = classificador()
optimizer = torch.optim.Adam(net.parameters())

In [ ]:
from google.colab import drive
drive.mount('/content/qdrive')


In [ ]:
path = '/content/qdrive/MyDrive/dataset.zip'
zip_object = zipfile.ZipFile(file = path, mode = 'r')
zip_object.extractall('./')
zip_object.close()

In [ ]:
data_dir_train = '/content/dataset/training_set'
data_dir_test = '/content/dataset/test_set'

In [ ]:
transform_train = transforms.Compose([
    transforms.Resize([64,64]),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(
        degrees = 7,
        translate = (0, 0.07),
        shear = 0.2,
        scale = (1, 1.2)
    ),
    transforms.ToTensor()
])
transform_test = transforms.Compose([
    transforms.Resize([64,64]),
    transforms.ToTensor()
])

In [ ]:
train_dataset = datasets.ImageFolder(data_dir_train, transform = transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 32, shuffle = True)
test_dataset = datasets.ImageFolder(data_dir_test, transform = transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 32, shuffle = True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
net.to(device)

In [ ]:
def training_loop(loader, epoch):
  running_loss = 0.
  running_accuracy = 0.

  for i, data in enumerate(loader):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = net.forward(inputs)

    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    predicted = torch.tensor([1 if output > 0.5 else 0 for output in outputs]).to(device)

    equals = predicted == labels.view(*predicted.shape)

    accuracy = torch.mean(equals.float())
    running_accuracy += accuracy

  print('\rEpoch {:3d} FINALIZADA: perda {:.5f} - precisao {:.5f}'.format(epoch + 1, running_loss/len(loader), running_accuracy/len(loader)) )

In [ ]:
for epoch in range(10):
  print ('Treinando...')
  training_loop(train_loader, epoch)
  net.eval()
  print('Validando...')
  training_loop(test_loader, epoch)
  net.train()